In [ ]:
from pathlib import Path
import polars as pl
import numpy as np

from ebrec.utils._constants import *
from ebrec.evaluation.beyond_accuracy import (
    IntralistDiversity,
    Distribution,
    Serendipity,
    Sentiment,
    Coverage,
    Novelty,
)
from ebrec.utils._articles import create_sort_based_prediction_score
from ebrec.utils._behaviors import truncate_history
from ebrec.utils._polars import slice_join_dataframes
from ebrec.utils._python import (
    rank_predictions_by_score,
    write_submission_file,
    write_json_file,
    read_json_file,
)


In [ ]:
ROOT_FOLDER = "evaluation_artifacts"
DATASET_SIZE = "demo"
# ROOT PATH:
PATH = Path(f"../../data/processed/")

## We are using the LARGE articles dataset to ensure we have all articles IDs,
# for beyond-accuracy; as only 154 aids are found in the demo.
ARTICLES_PATH = Path(f"../../data/articles_large_only/")

# PATH TO DUMP ARTIFACTS:
PATH_BEYOND_ACCURACY = Path(f"../npa/artifacts")
PATH_BEYOND_ACCURACY.mkdir(exist_ok=True, parents=True)
# BASELINE ARTIFACTS:
PATH_BEYOND_ACCURACY_BASELINES = PATH_BEYOND_ACCURACY.joinpath("baselines")
PATH_BEYOND_ACCURACY_BASELINES.mkdir(parents=True, exist_ok=True)

BEYOND_ACCURACY_HISTORY_DICT = "beyond_accuracy_history_dict.json"
BEYOND_ACCURACY_USERS_DICT = "beyond_accuracy_users_dict.json"
CANDIDATE_LIST = "candidate_list.json"
ARTICLES_DICT = "articles_dict.json"
BEHAVIORS_TIMESTAMP_DICT = "behaviors_timestamp_dict.json"
#
BASELINE_DIVERSITY = "intralist_diversity.json"
BASELINE_SENTIMENT_SCORE = "sentiment_score.json"
BASELINE_NOVELTY = "novelty.json"
BASELINE_SERENDIPITY = "serendipity.json"
BASELINE_COVERAGE = "coverage.json"
BASELINE_DISTRIBUTION_CATEGORY = "distribution_category.json"
BASELINE_DISTRIBUTION_SENTIMENT_LABEL = "distribution_sentiment_label.json"
BASELINE_DISTRIBUTION_TOPICS = "distribution_topics.json"


In [ ]:
# df_train_articles = pl.scan_parquet(data_base / "train_articles.parquet")
# df_test_articles = pl.scan_parquet(data_base / "test_articles.parquet")

# Load data:
df_beyond_accuracy = pl.scan_parquet(PATH.joinpath("test.parquet")).filter(
    pl.col("is_beyond_accuracy")
)
df_behaviors = pl.scan_parquet(PATH.joinpath("test.parquet")).filter(
    ~pl.col("is_beyond_accuracy")
)
df_articles = pl.scan_parquet(ARTICLES_PATH.joinpath("articles.parquet"))
df_history = pl.scan_parquet(PATH.joinpath("test.parquet")).select(
    DEFAULT_USER_COL, DEFAULT_HISTORY_ARTICLE_ID_COL
)

In [ ]:
df_beyond_accuracy.head(2)

In [ ]:
candidate_list = (
    df_beyond_accuracy.select(pl.col(DEFAULT_INVIEW_ARTICLES_COL).first())
    .collect()
    .to_series()
)[0].to_list()


write_json_file(
    candidate_list, PATH_BEYOND_ACCURACY.joinpath(CANDIDATE_LIST), verbose=True
)

print(f"Number of Candidate IDs: {len(candidate_list)} (example: {candidate_list[:5]})")

In [ ]:
import numpy as np
# Check if the lists are identical
# Read the candidate list back from the JSON file
load_candidate_list = read_json_file(PATH_BEYOND_ACCURACY.joinpath(CANDIDATE_LIST))

# Check if the lists are identical
if candidate_list != load_candidate_list:
    print("Mismatched entries:")
    
    # Iterate through both lists and print mismatches
    mismatched_indices = []
    min_length = min(len(candidate_list), len(load_candidate_list))
    
    for i in range(min_length):
        if candidate_list[i] != load_candidate_list[i]:
            print(f"Index {i}: candidate_list = {candidate_list[i]}, load_candidate_list = {load_candidate_list[i]}")
            mismatched_indices.append(i)
    
    # Check for additional elements in either list
    if len(candidate_list) > min_length:
        print(f"Extra entries in candidate_list starting from index {min_length}: {candidate_list[min_length:]}")
        mismatched_indices.extend(range(min_length, len(candidate_list)))
    
    if len(load_candidate_list) > min_length:
        print(f"Extra entries in load_candidate_list starting from index {min_length}: {load_candidate_list[min_length:]}")
        mismatched_indices.extend(range(min_length, len(load_candidate_list)))
    
    raise ValueError(f"candidate_list and load_candidate_list do not match at indices {mismatched_indices}")
else:
    print("Sanity check - passed")




In [ ]:
user_meta_columns = [
    DEFAULT_IS_SUBSCRIBER_COL,
    DEFAULT_IS_SSO_USER_COL,
    DEFAULT_POSTCODE_COL,
    DEFAULT_GENDER_COL,
    DEFAULT_AGE_COL,
]
df_users = df_beyond_accuracy.select(pl.col(user_meta_columns)).collect()

users_dict = {col: df_users[col].to_list() for col in df_users.columns}
write_json_file(
    users_dict, PATH_BEYOND_ACCURACY.joinpath(BEYOND_ACCURACY_USERS_DICT), verbose=True
)
print(f"#rows: {df_users.shape[0]}")
df_users.head(3)

In [ ]:
HISTORY_SIZE = 20
df_history_truncate = df_history.pipe(
    truncate_history,
    column=DEFAULT_HISTORY_ARTICLE_ID_COL,
    history_size=HISTORY_SIZE,
    padding_value=None,
    enable_warning=False,
)
# =>
df_user_histoies = (
    df_beyond_accuracy.select(DEFAULT_USER_COL)
    .join(df_history_truncate, on=DEFAULT_USER_COL, how="left")
    .collect()
)
user_history_dict = {
    DEFAULT_HISTORY_ARTICLE_ID_COL: df_user_histoies[
        DEFAULT_HISTORY_ARTICLE_ID_COL
    ].to_list()
}
write_json_file(
    user_history_dict,
    PATH_BEYOND_ACCURACY.joinpath(BEYOND_ACCURACY_HISTORY_DICT),
    verbose=True,
)
print(f"#rows: {df_user_histoies.shape[0]}")
df_user_histoies.head(3)

In [ ]:
df_behaviors_timestamp = df_behaviors.select(
    pl.col(DEFAULT_IMPRESSION_TIMESTAMP_COL).cast(pl.Utf8),
).collect()
behaviors_timestamp_dict = {
    DEFAULT_IMPRESSION_TIMESTAMP_COL: df_behaviors_timestamp[
        DEFAULT_IMPRESSION_TIMESTAMP_COL
    ].to_list()
}
write_json_file(
    behaviors_timestamp_dict,
    PATH_BEYOND_ACCURACY.joinpath(BEHAVIORS_TIMESTAMP_DICT),
    verbose=True,
)
print(f"#rows: {df_behaviors_timestamp.shape[0]}")
df_behaviors_timestamp.head(3)

In [ ]:
history_article_id = (
    df_user_histoies.lazy()
    .select(pl.col(DEFAULT_HISTORY_ARTICLE_ID_COL).explode().unique())
    .collect()[DEFAULT_HISTORY_ARTICLE_ID_COL]
    .to_list()
)
print(f"#history_article_id: {len(history_article_id)})")

In [80]:
aids_in_split = (
    df_articles.select(DEFAULT_ARTICLE_ID_COL)
    .collect()[DEFAULT_ARTICLE_ID_COL]
    .to_list()
)

history_article_id = [id for id in history_article_id if id in aids_in_split]
candidate_list = [id for id in candidate_list if id in aids_in_split]

article_ids = candidate_list + history_article_id
print(
    f"#articles: {len(article_ids)} (#candidate_list: {len(candidate_list)} & #history_article_id: {len(history_article_id)})"
)

#articles: 2072 (#candidate_list: 250 & #history_article_id: 1822)


In [81]:
df_lookup_articles = (
    df_articles.filter(pl.col(DEFAULT_ARTICLE_ID_COL).is_in(article_ids))
    .with_columns(
        pl.col(
            DEFAULT_ARTICLE_MODIFIED_TIMESTAMP_COL,
            DEFAULT_ARTICLE_PUBLISHED_TIMESTAMP_COL,
        ).cast(pl.Utf8)
    )
    # Zeros might cause issues
    .with_columns(
        pl.col(DEFAULT_TOTAL_INVIEWS_COL, DEFAULT_TOTAL_PAGEVIEWS_COL).fill_null(1)
    )
    .collect()
)
print(f"df_lookup_articles shape: {df_lookup_articles.shape}")

df_lookup_articles shape: (2072, 21)


In [82]:
DEFAULT_TOTAL_PAGEVIEWS_COL_NORMALIZED_MAX = (
    DEFAULT_TOTAL_PAGEVIEWS_COL + "_normalized_max"
)
DEFAULT_TOTAL_PAGEVIEWS_COL_NORMALIZED_MIN_MAX = (
    DEFAULT_TOTAL_PAGEVIEWS_COL + "_normalized_min_max"
)

MIN_X = df_lookup_articles[DEFAULT_TOTAL_PAGEVIEWS_COL].min()
MAX_X = df_lookup_articles[DEFAULT_TOTAL_PAGEVIEWS_COL].max()
MIN_RANGE = 1e-4
MAX_RANGE = 1.0

df_lookup_articles = df_lookup_articles.with_columns(
    (  # SIMPLE MAX NORMALIZATION: x / max()
        pl.col(DEFAULT_TOTAL_PAGEVIEWS_COL) / pl.col(DEFAULT_TOTAL_PAGEVIEWS_COL).max()
    ).alias(DEFAULT_TOTAL_PAGEVIEWS_COL_NORMALIZED_MAX)
).with_columns(
    (  #  MIN-MAX NORMALIZATION: ( x_i − X_min ⁡ ) / ( X_max ⁡ − X_min ⁡ ) * (max_range − min_range) + min_range
        ((pl.col(DEFAULT_TOTAL_PAGEVIEWS_COL) - MIN_X) / (MAX_X - MIN_X))
        * (MAX_RANGE - MIN_RANGE)
        + MIN_RANGE
    ).alias(
        DEFAULT_TOTAL_PAGEVIEWS_COL_NORMALIZED_MIN_MAX
    )
)

df_lookup_articles.select(
    DEFAULT_TOTAL_PAGEVIEWS_COL_NORMALIZED_MAX,
    DEFAULT_TOTAL_PAGEVIEWS_COL_NORMALIZED_MIN_MAX,
).describe()

shape: (9, 3)
┌────────────┬────────────────────────────────┬───────────────────────────────────┐
│ statistic  ┆ total_pageviews_normalized_max ┆ total_pageviews_normalized_min_m… │
│ ---        ┆ ---                            ┆ ---                               │
│ str        ┆ f64                            ┆ f64                               │
╞════════════╪════════════════════════════════╪═══════════════════════════════════╡
│ count      ┆ 2072.0                         ┆ 2072.0                            │
│ null_count ┆ 0.0                            ┆ 0.0                               │
│ mean       ┆ 0.096182                       ┆ 0.096271                          │
│ std        ┆ 0.080474                       ┆ 0.080466                          │
│ min        ┆ 0.000001                       ┆ 0.0001                            │
│ 25%        ┆ 0.040181                       ┆ 0.040276                          │
│ 50%        ┆ 0.082642                       ┆ 0.082732                          │
│ 75%        ┆ 0.137439                       ┆ 0.137524                          │
│ max        ┆ 1.0                            ┆ 1.0                               │
└────────────┴────────────────────────────────┴───────────────────────────────────┘

In [97]:
# => Embeddings:
BERT_VECTOR = "bert_base_multilingual_cased"
ROBERTA_VECTOR = "xlm_roberta_base"
import os
CONTRASTIVE_VECTOR = "contrastive_vector"
DOCUMENT_VECTOR = "document_vector"


def load_join_embeddings(df: pl.DataFrame, emb_path: Path) -> pl.DataFrame:
    emb_contrastive = (
        pl.scan_parquet(PATH.parent.joinpath(emb_path))
        .filter(pl.col(DEFAULT_ARTICLE_ID_COL).is_in(df.select(DEFAULT_ARTICLE_ID_COL)))
        .collect()
    )
    return df.join(emb_contrastive, on=DEFAULT_ARTICLE_ID_COL, how="left")

print(os.path.exists(f"../Ekstra_Bladet_contrastive_vector/Ekstra_Bladet_contrastive_vector/{CONTRASTIVE_VECTOR}.parquet"))

df_lookup_articles = df_lookup_articles.pipe(
    load_join_embeddings,
    emb_path=os.path.abspath(f"../../data/Ekstra_Bladet_contrastive_vector/Ekstra_Bladet_contrastive_vector/{CONTRASTIVE_VECTOR}.parquet"),
).pipe(
    load_join_embeddings,
    emb_path=os.path.abspath(f"../../data/Ekstra_Bladet_word2vec/Ekstra_Bladet_word2vec/{DOCUMENT_VECTOR}.parquet"),
)
print(f"#rows: {df_lookup_articles.shape[0]}")
df_lookup_articles.head(2)

False
#rows: 2072


shape: (2, 25)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ article_i ┆ title     ┆ subtitle  ┆ last_modi ┆ … ┆ total_pag ┆ total_pag ┆ contrasti ┆ document │
│ d         ┆ ---       ┆ ---       ┆ fied_time ┆   ┆ eviews_no ┆ eviews_no ┆ ve_vector ┆ _vector  │
│ ---       ┆ str       ┆ str       ┆ ---       ┆   ┆ rmalized_ ┆ rmalized_ ┆ ---       ┆ ---      │
│ i32       ┆           ┆           ┆ str       ┆   ┆ max       ┆ min_m…    ┆ list[f32] ┆ list[f32 │
│           ┆           ┆           ┆           ┆   ┆ ---       ┆ ---       ┆           ┆ ]        │
│           ┆           ┆           ┆           ┆   ┆ f64       ┆ f64       ┆           ┆          │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ 3039288   ┆ Rigmandsd ┆ Deirdre   ┆ 2023-06-2 ┆ … ┆ 0.000001  ┆ 0.0001    ┆ [-0.00557 ┆ [0.06260 │
│           ┆ atter     ┆ Husted-An ┆ 9 06:21:0 ┆   ┆           ┆           ┆ 3,        ┆ 9, -0.01 │
│           ┆ gift      ┆ dersen    ┆ 3.000000  ┆   ┆           ┆           ┆ 0.043804, ┆ 8904, …  │
│           ┆           ┆ har fund… ┆           ┆   ┆           ┆           ┆ …         ┆ 0.01272] │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆ 0.011333… ┆          │
│ 3083144   ┆ Mor og    ┆ To        ┆ 2023-06-2 ┆ … ┆ 0.000001  ┆ 0.0001    ┆ [-0.01230 ┆ [0.01061 │
│           ┆ datter    ┆ kvinder,  ┆ 9 06:21:5 ┆   ┆           ┆           ┆ 8,        ┆ 7, -0.03 │
│           ┆ stjal     ┆ der udgav ┆ 7.000000  ┆   ┆           ┆           ┆ 0.147319, ┆ 9497, …  │
│           ┆ hest      ┆ sig for   ┆           ┆   ┆           ┆           ┆ …         ┆ 0.023572 │
│           ┆           ┆ at…       ┆           ┆   ┆           ┆           ┆ -0.00227… ┆ …        │
└───────────┴───────────┴───────────┴───────────┴───┴───────────┴───────────┴───────────┴──────────┘

In [98]:
articles_dict = {}
for row in df_lookup_articles.iter_rows(named=True):
    # Note, all keys in dictionaries are converted to strings, when serializing an object to JSON format.
    articles_dict[str(row[DEFAULT_ARTICLE_ID_COL])] = row
# Write it:
write_json_file(
    articles_dict, PATH_BEYOND_ACCURACY.joinpath(ARTICLES_DICT), verbose=True
)
print(f"#articles: {len(articles_dict)}")

Writing JSON: '..\npa\artifacts\articles_dict.json'
#articles: 2072


In [99]:
def n_items(d) -> int:
    return len(d[list(d)[0]])


# =>
behaviors_timestamp_dict = read_json_file(
    PATH_BEYOND_ACCURACY.joinpath(BEHAVIORS_TIMESTAMP_DICT)
)
print(f"#behaviors_timestamp_dict: {n_items(behaviors_timestamp_dict)}")

# =>
history_dict = read_json_file(
    PATH_BEYOND_ACCURACY.joinpath(BEYOND_ACCURACY_HISTORY_DICT)
)
print(
    f"#history_dict: {n_items(history_dict)}\n history_dict.keys(): {history_dict.keys()}"
)

# =>
users_dict = read_json_file(PATH_BEYOND_ACCURACY.joinpath(BEYOND_ACCURACY_USERS_DICT))
print(f"#users_dict {n_items(users_dict)}\n users_dict.keys(): {users_dict.keys()}")

# =>
user_history_dict = read_json_file(
    PATH_BEYOND_ACCURACY.joinpath(BEYOND_ACCURACY_HISTORY_DICT)
)
print(
    f"#user_history_dict {n_items(user_history_dict)}\n users_dict.keys(): {user_history_dict.keys()}"
)

# =>
articles_dict = {
    int(key): val
    for key, val in read_json_file(PATH_BEYOND_ACCURACY.joinpath(ARTICLES_DICT)).items()
}
aid_keys = articles_dict[list(articles_dict)[0]].keys()
print(f"#articles_dict: {len(articles_dict)}\n articles_dict[ID].keys(): {aid_keys}")

# => Only the once actually found in the dataset (for demo only 154 of 250 are represent)
candidate_list = [
    id
    for id in read_json_file(PATH_BEYOND_ACCURACY.joinpath(CANDIDATE_LIST))
    if id in list(articles_dict)
]
print(f"#candidate_list: {len(candidate_list)}")

df_candidate_articles = df_lookup_articles.filter(
    pl.col(DEFAULT_ARTICLE_ID_COL).is_in(candidate_list)
)
print(f"#candidate-articles (df): {df_candidate_articles.shape[0]}")

#behaviors_timestamp_dict: 133369
#history_dict: 2363
 history_dict.keys(): dict_keys(['article_id_fixed'])
#users_dict 1998
 users_dict.keys(): dict_keys(['is_subscriber', 'is_sso_user', 'postcode', 'gender', 'age'])
#user_history_dict 2363
 users_dict.keys(): dict_keys(['article_id_fixed'])
#articles_dict: 2072
 articles_dict[ID].keys(): dict_keys(['article_id', 'title', 'subtitle', 'last_modified_time', 'premium', 'body', 'published_time', 'image_ids', 'article_type', 'url', 'ner_clusters', 'entity_groups', 'topics', 'category', 'subcategory', 'category_str', 'total_inviews', 'total_pageviews', 'total_read_time', 'sentiment_score', 'sentiment_label', 'total_pageviews_normalized_max', 'total_pageviews_normalized_min_max', 'contrastive_vector', 'document_vector'])
#candidate_list: 250
#candidate-articles (df): 250


In [100]:
df_candidates_editorial_picks = create_sort_based_prediction_score(
    df_candidate_articles,
    column=DEFAULT_TOTAL_INVIEWS_COL,
    desc=True,
)
candidates_editorial_picks = np.array(
    [df_candidates_editorial_picks.select(DEFAULT_ARTICLE_ID_COL).to_series()]
)
# =>
print(candidates_editorial_picks[:, :2])
df_candidates_editorial_picks.head(2)

[[9790335 9791587]]


shape: (2, 3)
┌────────────┬───────────────┬──────────────────┐
│ article_id ┆ total_inviews ┆ prediction_score │
│ ---        ┆ ---           ┆ ---              │
│ i32        ┆ i32           ┆ f64              │
╞════════════╪═══════════════╪══════════════════╡
│ 9790335    ┆ 1698890       ┆ 1.0              │
│ 9791587    ┆ 1369829       ┆ 0.5              │
└────────────┴───────────────┴──────────────────┘

In [101]:
df_candidates_popular = create_sort_based_prediction_score(
    df_candidate_articles,
    column=DEFAULT_TOTAL_PAGEVIEWS_COL,
    desc=True,
)
candidates_popular = np.array(
    [df_candidates_popular.select(DEFAULT_ARTICLE_ID_COL).to_series()]
)
# =>
print(candidates_popular[:, :2])
df_candidates_popular.head(2)

[[9791428 9792719]]


shape: (2, 3)
┌────────────┬─────────────────┬──────────────────┐
│ article_id ┆ total_pageviews ┆ prediction_score │
│ ---        ┆ ---             ┆ ---              │
│ i32        ┆ i32             ┆ f64              │
╞════════════╪═════════════════╪══════════════════╡
│ 9791428    ┆ 256541          ┆ 1.0              │
│ 9792719    ┆ 209050          ┆ 0.5              │
└────────────┴─────────────────┴──────────────────┘

In [102]:
intralist_diversity = IntralistDiversity()
distribution = Distribution()
serendipity = Serendipity()
sentiment = Sentiment()
coverage = Coverage()
novelty = Novelty()

In [104]:
RANDOM_ITER = df_beyond_accuracy.select(DEFAULT_INVIEW_ARTICLES_COL).collect().shape[0]
TOP_N = 5

np.random.seed(123)
# Make list:
top_n_candidates_random = [
    np.random.choice(candidate_list, size=TOP_N, replace=False)
    for _ in range(RANDOM_ITER)
]
top_n_candidates_editorial_picks = candidates_editorial_picks[:, :TOP_N]
top_n_candidates_popular = candidates_popular[:, :TOP_N]
#
# Set them as tuples, just to loop through it:
candidates_name_pairs = [
    [top_n_candidates_editorial_picks, "top-inview-articles"],
    [top_n_candidates_popular, "popular"],
    [top_n_candidates_random, "random"],
]
# =>
user_history = user_history_dict[DEFAULT_HISTORY_ARTICLE_ID_COL]

print(f"#random-iterations: {RANDOM_ITER}")
print(f"Top@{TOP_N} ranked articles")

#random-iterations: 1998
Top@5 ranked articles


In [105]:
##add predictions from model

candidates_your_model = [
    np.random.choice(candidate_list, size=TOP_N, replace=False)
    for _ in range(RANDOM_ITER)
]
## Just uncomment and add the prediction and name:
# candidates_name_pairs.append([candidates_your_model, "random_2"])

In [106]:
intralist_diversity_dict = {"name": ["mean", "std"]}

for candidates, list_name in candidates_name_pairs:
    scores = intralist_diversity(
        candidates,
        lookup_dict=articles_dict,
        lookup_key=CONTRASTIVE_VECTOR,
    )
    intralist_diversity_dict[f"{list_name}_{intralist_diversity.name}"] = [
        scores.mean(),
        scores.std(),
    ]

write_json_file(
    intralist_diversity_dict,
    PATH_BEYOND_ACCURACY_BASELINES.joinpath(BASELINE_DIVERSITY),
    verbose=True,
)

pl.DataFrame(intralist_diversity_dict)

Writing JSON: '..\npa\artifacts\baselines\intralist_diversity.json'


shape: (2, 4)
┌──────┬────────────────────────────────┬─────────────────────────────┬────────────────────────────┐
│ name ┆ top-inview-articles_intralist_ ┆ popular_intralist_diversity ┆ random_intralist_diversity │
│ ---  ┆ di…                            ┆ ---                         ┆ ---                        │
│ str  ┆ ---                            ┆ f64                         ┆ f64                        │
│      ┆ f64                            ┆                             ┆                            │
╞══════╪════════════════════════════════╪═════════════════════════════╪════════════════════════════╡
│ mean ┆ 0.790542                       ┆ 0.840236                    ┆ 0.756229                   │
│ std  ┆ 0.0                            ┆ 0.0                         ┆ 0.09097                    │
└──────┴────────────────────────────────┴─────────────────────────────┴────────────────────────────┘

In [107]:
ROBERTA_EMB = "FacebookAI/xlm-roberta-base"
BERT_EMB = "google-bert/bert-base-multilingual-cased"
print(
    f"{CONTRASTIVE_VECTOR}: {intralist_diversity(top_n_candidates_editorial_picks, lookup_dict=articles_dict, lookup_key=CONTRASTIVE_VECTOR).mean()}"
)
print(
    f"{DOCUMENT_VECTOR}: {intralist_diversity(top_n_candidates_editorial_picks, lookup_dict=articles_dict, lookup_key=DOCUMENT_VECTOR).mean()}"
)

contrastive_vector: 0.7905415595169083
document_vector: 0.1584846028291677


In [108]:
sentiment_dict = {"name": ["mean", "std"]}

for candidates, list_name in candidates_name_pairs:
    scores = sentiment(
        candidates,
        lookup_dict=articles_dict,
        lookup_key=DEFAULT_SENTIMENT_SCORE_COL,
    )
    sentiment_dict[f"{list_name}_{sentiment.name}"] = [
        scores.mean(),
        scores.std(),
    ]

write_json_file(
    sentiment_dict,
    PATH_BEYOND_ACCURACY_BASELINES.joinpath(BASELINE_SENTIMENT_SCORE),
    verbose=True,
)

pl.DataFrame(sentiment_dict)


Writing JSON: '..\npa\artifacts\baselines\sentiment_score.json'


shape: (2, 4)
┌──────┬───────────────────────────────┬───────────────────┬──────────────────┐
│ name ┆ top-inview-articles_sentiment ┆ popular_sentiment ┆ random_sentiment │
│ ---  ┆ ---                           ┆ ---               ┆ ---              │
│ str  ┆ f64                           ┆ f64               ┆ f64              │
╞══════╪═══════════════════════════════╪═══════════════════╪══════════════════╡
│ mean ┆ 0.7294                        ┆ 0.78342           ┆ 0.820744         │
│ std  ┆ 0.0                           ┆ 0.0               ┆ 0.072173         │
└──────┴───────────────────────────────┴───────────────────┴──────────────────┘

In [109]:
novelty_dict = {"name": ["mean", "std"]}

for candidates, list_name in candidates_name_pairs:
    scores = novelty(
        candidates,
        lookup_dict=articles_dict,
        lookup_key=DEFAULT_TOTAL_PAGEVIEWS_COL_NORMALIZED_MIN_MAX,
    )
    novelty_dict[f"{list_name}_{novelty.name}"] = [
        scores.mean(),
        scores.std(),
    ]

write_json_file(
    novelty_dict,
    PATH_BEYOND_ACCURACY_BASELINES.joinpath(BASELINE_NOVELTY),
    verbose=True,
)

pl.DataFrame(novelty_dict)

Writing JSON: '..\npa\artifacts\baselines\novelty.json'


shape: (2, 4)
┌──────┬─────────────────────────────┬─────────────────┬────────────────┐
│ name ┆ top-inview-articles_novelty ┆ popular_novelty ┆ random_novelty │
│ ---  ┆ ---                         ┆ ---             ┆ ---            │
│ str  ┆ f64                         ┆ f64             ┆ f64            │
╞══════╪═════════════════════════════╪═════════════════╪════════════════╡
│ mean ┆ 3.52556                     ┆ 1.968051        ┆ 7.710898       │
│ std  ┆ 0.0                         ┆ 0.0             ┆ 2.002683       │
└──────┴─────────────────────────────┴─────────────────┴────────────────┘

In [110]:
serendipity_dict = {"name": ["mean", "std"]}
for candidates, list_name in candidates_name_pairs:
    if len(candidates) == 1:
        candidates = np.tile(candidates, len(user_history)).reshape(-1, TOP_N)
    #
    scores = serendipity(
        candidates,
        H=user_history,
        lookup_dict=articles_dict,
        lookup_key=CONTRASTIVE_VECTOR,
    )
    serendipity_dict[f"{list_name}_{serendipity.name}"] = [scores.mean(), scores.std()]

write_json_file(
    serendipity_dict,
    PATH_BEYOND_ACCURACY_BASELINES.joinpath(BASELINE_SERENDIPITY),
    verbose=True,
)

pl.DataFrame(serendipity_dict)

ValueError: The lengths of 'R' and 'H' do not match (1998 != 2363).

In [111]:
def concat_distribution_dict(dict_: dict) -> dict:
    output_results = (
        pl.concat(
            [pl.DataFrame(val) for val in dict_.values()],
            how="diagonal",
        )
        .with_row_index(name="name")
        .with_columns(pl.Series(dict_.keys()).alias("name"))
    ).to_dict()
    return {key: val.to_list() for key, val in output_results.items()}

In [112]:
COLUMN = DEFAULT_CATEGORY_STR_COL
distribution_dict = {
    f"{list_name}_{novelty.name}": distribution(
        candidates,
        lookup_dict=articles_dict,
        lookup_key=COLUMN,
    )
    for candidates, list_name in candidates_name_pairs
}
# =>
distribution_category_dict = concat_distribution_dict(distribution_dict)

write_json_file(
    distribution_category_dict,
    PATH_BEYOND_ACCURACY_BASELINES.joinpath(BASELINE_DISTRIBUTION_CATEGORY),
    verbose=True,
)

pl.DataFrame(distribution_category_dict)

Writing JSON: '..\npa\artifacts\baselines\distribution_category.json'


shape: (3, 11)
┌──────────────┬──────────┬──────────┬──────────┬───┬──────────┬─────────────┬──────────┬──────────┐
│ name         ┆ forbrug  ┆ nyheder  ┆ krimi    ┆ … ┆ musik    ┆ underholdni ┆ nationen ┆ incoming │
│ ---          ┆ ---      ┆ ---      ┆ ---      ┆   ┆ ---      ┆ ng          ┆ ---      ┆ ---      │
│ str          ┆ f64      ┆ f64      ┆ f64      ┆   ┆ f64      ┆ ---         ┆ f64      ┆ f64      │
│              ┆          ┆          ┆          ┆   ┆          ┆ f64         ┆          ┆          │
╞══════════════╪══════════╪══════════╪══════════╪═══╪══════════╪═════════════╪══════════╪══════════╡
│ top-inview-a ┆ 0.2      ┆ 0.2      ┆ 0.2      ┆ … ┆ null     ┆ null        ┆ null     ┆ null     │
│ rticles_nove ┆          ┆          ┆          ┆   ┆          ┆             ┆          ┆          │
│ lty          ┆          ┆          ┆          ┆   ┆          ┆             ┆          ┆          │
│ popular_nove ┆ null     ┆ 0.2      ┆ 0.2      ┆ … ┆ null     ┆ null        ┆ null     ┆ null     │
│ lty          ┆          ┆          ┆          ┆   ┆          ┆             ┆          ┆          │
│ random_novel ┆ 0.010711 ┆ 0.160561 ┆ 0.095896 ┆ … ┆ 0.035736 ┆ 0.082683    ┆ 0.024625 ┆ 0.004404 │
│ ty           ┆          ┆          ┆          ┆   ┆          ┆             ┆          ┆          │
└──────────────┴──────────┴──────────┴──────────┴───┴──────────┴─────────────┴──────────┴──────────┘

In [113]:
COLUMN = DEFAULT_SENTIMENT_LABEL_COL
distribution_dict = {
    f"{list_name}_{novelty.name}": distribution(
        candidates,
        lookup_dict=articles_dict,
        lookup_key=COLUMN,
    )
    for candidates, list_name in candidates_name_pairs
}
# =>
distribution_sentiment_dict = concat_distribution_dict(distribution_dict)

write_json_file(
    distribution_sentiment_dict,
    PATH_BEYOND_ACCURACY_BASELINES.joinpath(BASELINE_DISTRIBUTION_SENTIMENT_LABEL),
    verbose=True,
)

pl.DataFrame(distribution_sentiment_dict)

Writing JSON: '..\npa\artifacts\baselines\distribution_sentiment_label.json'


shape: (3, 4)
┌─────────────────────────────┬──────────┬──────────┬──────────┐
│ name                        ┆ Negative ┆ Positive ┆ Neutral  │
│ ---                         ┆ ---      ┆ ---      ┆ ---      │
│ str                         ┆ f64      ┆ f64      ┆ f64      │
╞═════════════════════════════╪══════════╪══════════╪══════════╡
│ top-inview-articles_novelty ┆ 0.4      ┆ 0.6      ┆ null     │
│ popular_novelty             ┆ 0.2      ┆ 0.4      ┆ 0.4      │
│ random_novelty              ┆ 0.393694 ┆ 0.318519 ┆ 0.287788 │
└─────────────────────────────┴──────────┴──────────┴──────────┘

In [114]:
COLUMN = DEFAULT_TOPICS_COL
distribution_dict = {
    f"{list_name}_{novelty.name}": distribution(
        candidates,
        lookup_dict=articles_dict,
        lookup_key=COLUMN,
    )
    for candidates, list_name in candidates_name_pairs
}
# =>

distribution_topics_dict = concat_distribution_dict(distribution_dict)

write_json_file(
    distribution_topics_dict,
    PATH_BEYOND_ACCURACY_BASELINES.joinpath(BASELINE_DISTRIBUTION_TOPICS),
    verbose=True,
)

pl.DataFrame(distribution_topics_dict)

Writing JSON: '..\npa\artifacts\baselines\distribution_topics.json'


shape: (3, 61)
┌────────────┬──────────┬──────────┬────────────┬───┬───────────┬───────────┬──────────┬───────────┐
│ name       ┆ Livsstil ┆ Sundhed  ┆ Sygdom og  ┆ … ┆ Kunstig   ┆ Forbruger ┆ Værdier  ┆ Offentlig │
│ ---        ┆ ---      ┆ ---      ┆ behandling ┆   ┆ intellige ┆ elektroni ┆ ---      ┆ transport │
│ str        ┆ f64      ┆ f64      ┆ ---        ┆   ┆ ns og     ┆ k         ┆ f64      ┆ ---       │
│            ┆          ┆          ┆ f64        ┆   ┆ software  ┆ ---       ┆          ┆ f64       │
│            ┆          ┆          ┆            ┆   ┆ ---       ┆ f64       ┆          ┆           │
│            ┆          ┆          ┆            ┆   ┆ f64       ┆           ┆          ┆           │
╞════════════╪══════════╪══════════╪════════════╪═══╪═══════════╪═══════════╪══════════╪═══════════╡
│ top-inview ┆ 0.133333 ┆ 0.066667 ┆ 0.066667   ┆ … ┆ null      ┆ null      ┆ null     ┆ null      │
│ -articles_ ┆          ┆          ┆            ┆   ┆           ┆           ┆          ┆           │
│ novelty    ┆          ┆          ┆            ┆   ┆           ┆           ┆          ┆           │
│ popular_no ┆ null     ┆ null     ┆ null       ┆ … ┆ null      ┆ null      ┆ null     ┆ null      │
│ velty      ┆          ┆          ┆            ┆   ┆           ┆           ┆          ┆           │
│ random_nov ┆ 0.017724 ┆ 0.009087 ┆ 0.008021   ┆ … ┆ 0.001907  ┆ 0.001094  ┆ 0.002804 ┆ 0.001206  │
│ elty       ┆          ┆          ┆            ┆   ┆           ┆           ┆          ┆           │
└────────────┴──────────┴──────────┴────────────┴───┴───────────┴───────────┴──────────┴───────────┘

In [115]:
np.random.seed(123)
df = (
    pl.concat([df_behaviors, df_beyond_accuracy])
    .select(DEFAULT_IMPRESSION_ID_COL, DEFAULT_INVIEW_ARTICLES_COL)
    .with_columns(
        pl.col(DEFAULT_INVIEW_ARTICLES_COL).map_elements(
            lambda x: list(np.random.permutation(len(x)) + 1)
        )
    )
    .collect()
)
df.head(3)

shape: (3, 2)
┌───────────────┬────────────────────┐
│ impression_id ┆ article_ids_inview │
│ ---           ┆ ---                │
│ u32           ┆ list[i32]          │
╞═══════════════╪════════════════════╡
│ 491367172     ┆ [8, 1, … 3]        │
│ 282300621     ┆ [4, 6, … 2]        │
│ 130618715     ┆ [5, 1, … 2]        │
└───────────────┴────────────────────┘

In [116]:
rank_score = df[:100].with_columns(
    pl.col(DEFAULT_INVIEW_ARTICLES_COL).list.eval(1 / pl.element())
)
rank_score.head(5)

shape: (5, 2)
┌───────────────┬──────────────────────────┐
│ impression_id ┆ article_ids_inview       │
│ ---           ┆ ---                      │
│ u32           ┆ list[f64]                │
╞═══════════════╪══════════════════════════╡
│ 491367172     ┆ [0.125, 1.0, … 0.333333] │
│ 282300621     ┆ [0.25, 0.166667, … 0.5]  │
│ 130618715     ┆ [0.2, 1.0, … 0.5]        │
│ 493717111     ┆ [0.333333, 0.5, … 1.0]   │
│ 131692548     ┆ [0.04, 0.058824, … 0.25] │
└───────────────┴──────────────────────────┘

In [117]:
rank_score.with_columns(
    pl.col(DEFAULT_INVIEW_ARTICLES_COL)
    .map_elements(lambda x: list(rank_predictions_by_score(x)))
    .alias("prediction_scores")
).head(5)

shape: (5, 3)
┌───────────────┬──────────────────────────┬───────────────────┐
│ impression_id ┆ article_ids_inview       ┆ prediction_scores │
│ ---           ┆ ---                      ┆ ---               │
│ u32           ┆ list[f64]                ┆ list[i64]         │
╞═══════════════╪══════════════════════════╪═══════════════════╡
│ 491367172     ┆ [0.125, 1.0, … 0.333333] ┆ [8, 1, … 3]       │
│ 282300621     ┆ [0.25, 0.166667, … 0.5]  ┆ [4, 6, … 2]       │
│ 130618715     ┆ [0.2, 1.0, … 0.5]        ┆ [5, 1, … 2]       │
│ 493717111     ┆ [0.333333, 0.5, … 1.0]   ┆ [3, 2, … 1]       │
│ 131692548     ┆ [0.04, 0.058824, … 0.25] ┆ [25, 17, … 4]     │
└───────────────┴──────────────────────────┴───────────────────┘

In [118]:
impression_ids = df[DEFAULT_IMPRESSION_ID_COL].to_list()
prediction_scores = df[DEFAULT_INVIEW_ARTICLES_COL].to_list()

write_submission_file(
    impression_ids=impression_ids,
    prediction_scores=prediction_scores,
    path=PATH_BEYOND_ACCURACY.joinpath("predictions.txt"),
    filename_zip=f"predictions_{DATASET_SIZE}_random.zip",
)

135367it [00:00, 191988.20it/s]


Zipping ..\npa\artifacts\predictions.txt to ..\npa\artifacts\predictions_demo_random.zip
